## A DETAILED DESIGN METHOD FOR PNEUMATIC TUBING SYSTEMS

Author: S. J. Gumley

Journal of Wind Engineering and Industrial Aerodynamics, vol 13, p 441-452, 1983.

In [ ]:
import Pkg
Pkg.activate(".")

In [ ]:
using SpecialFunctions
using PyPlot

In [ ]:
function lertabela(fname)
    ll = readlines(fname)
    nl = length(ll)
    nc = length(split(ll[1]))
    tab = zeros(nl, nc)
    for i = 1:nl
        s = split(ll[i])
        for k = 1:2
            tab[i,k] = parse(Float64, s[k])
        end
    end
    return tab
end




In [ ]:

function press_ratio(f, L::Vector{Float64}, D::Vector{Float64}, 
    V::Vector{Float64}; σ=0.0, γ=1.4, Pr=0.7, μ=1.5e-5, ρ=1.2, 
    k=1.4, a₀=343.2)
    
    ω = 2π*f
    
    N = length(L)
    
    α = zeros(Complex{Float64}, N)
    n = zeros(Complex{Float64}, N)
    ϕ = zeros(Complex{Float64}, N)
    t1 = zeros(Complex{Float64}, N)
    t2 = zeros(Complex{Float64}, N)
    t3 = zeros(Complex{Float64}, N)
    
    R = zeros(N)
    i32 = im*sqrt(im) #sqrt(-im)
    
    s = sqrt(Pr)
    for j = 1:N
        R[j] = D[j]/2
        α[j] = i32 * R[j]*sqrt(ω*ρ/μ)
        n[j] = (1 + (γ-1)/γ * besselj(2, α[j]*s)/besselj0(α[j]*s) )^(-1)
        ϕ[j] = ω/a₀ * sqrt(besselj0(α[j])/besselj(2, α[j])) * sqrt(γ/n[j])
    end
    
    r = zeros(Complex{Float64}, N)
    #r[N+1] = 
    
    
    for j = N:-1:1
        term1 = cosh(ϕ[j]*L[j])
        term2 = V[j]/(π*R[j]^2) * (σ + 1.0/k) * n[j] * ϕ[j] * sinh(ϕ[j]*L[j])
        if j==N
            term3 = 0.0*im
        else
            x1 = (R[j+1]/R[j])^2 
            x2 = (ϕ[j+1]/ϕ[j])
            x3 = (besselj0(α[j]) / besselj0(α[j+1]))
            x4 = (besselj(2, α[j+1]) / besselj(2, α[j]))
            x5 = sinh(ϕ[j]*L[j]) / sinh(ϕ[j+1]*L[j+1])
            
            x6 = cosh(ϕ[j+1]*L[j+1]) - 1.0/r[j+1]
            term3 = x1 * x2 * x3 * x4 * x5 * x6
            
        end
        #println(term3)
        t1[j] = term1
        t2[j] = term2
        t3[j] = term3
        r[j] = term1 + term2 + term3
    end
    
    p = 1.0 + 0.0*im
    
    for j = 1:N
        p *= 1/r[j]
    end
    
    return p
end

    

            
    

In [ ]:
function pratio_simple(f, L, D, V; σ=0.0, γ=1.4, Pr=0.7, μ=1.5e-5, ρ=1.09, k=1.4, a₀=343.2)

    ω = 2π*f
    R = D/2
    α = im*sqrt(im) * R * sqrt(ω*ρ/μ)
    s = α*sqrt(Pr)
    n = ( 1.0 + ((γ-1)/γ) * besselj(2, s) / besselj0(s)  )^(-1)
    ϕ = ω/a₀ * sqrt(besselj0(α)/besselj(2,α)) * sqrt(γ/n)
    r1 = cosh(ϕ*L)
    r2 = V/(π*R^2) * (σ + 1/k)*n*ϕ*sinh(ϕ*L)
    #return [r1,r2]
    return(1.0/(r1+r2))
end

In [ ]:
(press_ratio(100.0, [0.456], [1.5]*0.001*1, [69e-9]) -
pratio_simple(100.0, 0.456, 1.5*0.001*1, 69e-9))

In [ ]:
#L = [905, 15, 95] * 0.001 # [1.0]# 
#V = [0.0, 0.0, 0.0]# [0.0, 100.0] * 1e-9  # 
#D =  [1, 0.25, 1]*0.001
L = [1.0]
V = [0.0]
D = [0.0011]

In [ ]:
press_ratio(200.0, L, D, V)

In [ ]:
#ratio = lertabela("gum-rat.txt")
#phase = lertabela("gum-pha.txt");
ratio = lertabela("hol-rat.txt") #Figura 6 do Holmes
phase = lertabela("hol-pha.txt");

In [ ]:
function compute_angle(f::AbstractVector, r::AbstractVector)
    ϕ = angle.(r)
    np = length(f)
    
    s = 0.0
    for i = 2:np
        dϕ = (ϕ[i]+s) - ϕ[i-1]
        if dϕ > 6
            s -= 2π
        elseif dϕ < -6
            s += 2π
        end
        ϕ[i] += s
    end
    return ϕ
end

In [ ]:
nf = 800
f = 1.0:nf
#f = Float64[1:20:500]
r = zeros(Complex{Float64}, length(f))
r2 = zeros(Complex{Float64}, length(f))
V2 = [0.0, 200.0*1e-9]
for i = 1:length(f)
    
    #r[i] = press_ratio(f[i], [0.280, 0.006, 0.170], [1.5, 0.305, 1.5]*0.001, [0.0, 0.0, 69e-9]; σ=0.0)
    #r2[i] = press_ratio(f[i], [0.280, 0.007, 0.170], [1.5, 0.305, 1.5]*0.001, [0.0, 0.0, 69e-9]; σ=0.0)
    r[i] = press_ratio(f[i], [0.5], [1.5]*0.001*0.999, [257e-9]; σ=0.0)
    r2[i] = press_ratio(f[i], [0.5], [1.5]*0.001*0.95, [257e-9]; σ=0.0)
    #r2[i] = press_ratio(f[i], [0.5, 0.020], [1.5, 1.0]*0.001, [0.0, 107e-9]; σ=0.0)
    
    
    #r2[i]= press_ratio(f[i], [0.3, 0.02, 0.2], [0.001, 0.0003, 0.001], [0.0, 0.0, 0.0])
#    r2[i] = press_ratio(f[i], L, D, V2)
end
a =  compute_angle(f, r) .* 180/pi;
a2 = compute_angle(f, r2) .* 180/pi;





In [ ]:
subplot(2, 1, 1)
plot(f, abs.(r), "b-")
#plot(f, abs(r), "b-", f, abs.(r2), "g-", ratio[:,1], ratio[:,2], "r--")
subplot(2, 1, 2)
plot(f, a, "b-")

